In [13]:
#Expand notebook to take full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

#Jupyter magic to notify when a cell finishes execution with %%notify command -- does not work with Jupyterlab
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

###
import sys
sys.path.insert(0,'../../src/')

%load_ext autoreload
%autoreload 2

%pdb

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Automatic pdb calling has been turned ON


In [14]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.nn.modules import Module
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import copy
import pickle
import model_archs

from utils_plot import show_sws_weights, show_weights, print_dims, prune_plot, draw_sws_graphs, joint_plot, plot_data
from utils_model import test_accuracy, train_epoch, retrain_sws_epoch, model_prune, get_weight_penalty, layer_accuracy
from utils_misc import trueAfterN, logsumexp, root_dir, model_load_dir, get_ab
from utils_sws import GaussianMixturePrior, special_flatten, KL, compute_responsibilies, merger, sws_prune, sws_prune_l2
from mnist_loader import search_train_data, search_retrain_data, search_validation_data, train_data, test_data, batch_size
from extract_targets import get_targets
from retrain_layer import init_retrain_layer
from retrain_model import retrain_model

In [17]:
model_kd_s, gmp_kd_s, res_kd_s = retrain_model(500, 10, 2500, 1250, 1e-6, int(0), 15, "LeNet_300_100", "search", 'CESNT', "", True)

0-component Mean: 2.0 Variance: 0.0016
Non-zero component Mean: 50.0 Variance: 5.0
Epoch: 10. Test Accuracy: 98.09
Retrain Test: 98.09, Retrain Validation: 97.96, Prune Test: 26.80, Prune Validation: 25.99, Prune Sparsity: 98.06


In [18]:
res_kd_s

{'complexity_loss': [116197.140625,
  89452.2265625,
  67014.8046875,
  48664.34765625,
  34298.6171875,
  23687.884765625,
  16915.13671875,
  12602.951171875,
  9516.755859375,
  7136.04296875],
 'data_size': 'search',
 'epochs': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'final_weights': {'fc1': array([-3.0097100e-10, -3.0140135e-10, -3.0172287e-10, ...,
         -4.2337708e-02,  2.0152245e-01,  1.1859013e-02], dtype=float32),
  'fc2': array([ 2.3737418e-02,  2.2623736e-01, -3.6181422e-10, ...,
         -1.1424637e-01,  6.4659700e-02,  4.2199515e-02], dtype=float32),
  'fc3': array([ 1.9858105e-01, -4.0156418e-01, -2.4072081e-01, ...,
         -8.6184554e-02,  1.5870447e-01, -3.7376329e-04], dtype=float32)},
 'gmp_means': array([[-1.        , -0.84615386, -0.6923077 , -0.5384615 , -0.38461536,
         -0.23076922, -0.07692301,  0.07692313,  0.23076928,  0.38461542,
          0.53846157,  0.6923077 ,  0.846154  ,  1.        ],
        [-0.9981827 , -0.8445493 , -0.68936527, -0.53354955, -0.3